In [1]:
import os
import pandas as pd
from dataclasses import dataclass
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from ml_project import logger
from ml_project.utils.common import read_yaml, create_directories, save_json
from ml_project.constants import *
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [2]:
os.chdir("../")
!pwd

f:/Tutorials/Deep Learning/Machine Learning Projects/End to End Machine Learning/machine-learning-with-mlflow


In [3]:
dotenv_path = os.path.join('.env')
print('Dot Env Path: ' + str(dotenv_path))
load_dotenv(find_dotenv(filename=dotenv_path))

Dot Env Path: .env


True

In [4]:
# Entity for the Configuration form the config.yaml

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_url: str

In [5]:
# Configuration for the ModelEvaluationConfigurationManager

class ModelEvaluationConfigurationManager:
    def __init__(
        self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH,
        schema_path = SCHEMA_FILE_PATH
        ) -> None:
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)

        create_directories([self.config.artifacts_root])        
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.target_column

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_path = config.test_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_url = os.environ.get("MLFLOW_TRACKING_URI")
        )

        create_directories([config.root_dir])

        return model_evaluation_config

In [6]:
# Create the component for ModelEvaluation.

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, pred = predicted_qualities)

            # Saving metrics as lolcal

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are olther ways use the Model registry, which depends on the use case.
                # Please refer to the doc for more information.
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow

                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")


        

In [7]:
# Define the Pipeline.

try:
    config = ModelEvaluationConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as error:
    raise error
    

[2023-08-06 20:57:11,181: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2023-08-06 20:57:11,185: INFO: common: yaml file: params.yaml loaded successfully.]
[2023-08-06 20:57:11,190: INFO: common: yaml file: schema.yaml loaded successfully.]
[2023-08-06 20:57:11,193: INFO: common: Created directory at: artifacts]
[2023-08-06 20:57:11,195: INFO: common: Created directory at: artifacts/model_evaluation]
[2023-08-06 20:57:12,398: INFO: common: Json file save at: artifacts\model_evaluation\metrics.json]


c:\Python310\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2023/08/06 20:57:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.
